## Data Imputation 

## Stage 1: 
- preparing data 
- data imputation for 4 columns in the matrix 

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/data_clean.csv', sep=',', encoding='latin-1')
df = df.drop(columns=['SEQN', 'PAQ706'])
# print(df.head(), df.columns, df.shape)


In [2]:
df = df[df.BPXSY1.notnull()]

In [3]:
# regressing on this column 
df = df.drop(columns=['BPXSY1'])

In [4]:
# df = df[["BPXPLS", "BMXARMC", "bmi", "highbp" ]] # Reading 4 columns for now 
np_array = df.to_numpy(copy=True, na_value=np.nan) # Convert dataframe to numpy array 

To test our code, we will first delete values ourselves and compare output

## Stage 1A: 
- 1) Considering data with only non missing values (delete rows with any missing values)
- 2) Dropping values at random from this data 
- 3) Applying Low Rank Model estimation 
- 4) Evaluating the low rank estimate using true values 

In [5]:
## nonmissing_data is our ground truth, 
## MCAR stands for Missing completely at random; MCAR_data is to evaluate low rank model algorithm 

nonmissing_data = np_array[~np.isnan(np_array).any(axis=1), :] # delete rows with any missing values
MCAR_missing_indices = np.random.choice( # randomly select values from length ie flat array 
    len(nonmissing_data.flatten()), 
    size=int(len(nonmissing_data.flatten())/10)) 

MCAR_data = nonmissing_data
MCAR_data.flat[MCAR_missing_indices] = np.nan
print(MCAR_data) 

[[ 58.           1.           3.         ... 179.           0.
   33.55912298]
 [ 57.           2.           2.         ... 235.           0.
   28.97823892]
 [ 47.           1.           3.         ... 230.                  nan
   33.58048532]
 ...
 [ 47.           2.           4.         ... 141.           0.
           nan]
 [         nan   2.           4.         ... 241.           1.
   42.86189205]
 [ 43.                  nan   4.         ... 169.           0.
   50.0250699 ]]


## Low Rank Estimation 

In [6]:
## FIX: delete this line when no longer working with MCAR_data 
# np_array = MCAR_data 

## Get non-missing indices 
nonmissing_indices = np.argwhere(~np.isnan(np_array.flatten())).reshape(-1) 

## Create initial matrix (impute nans with zeroes) 
rating_matrix_ini = np.zeros(np_array.shape) - 1 
rating_matrix_ini.flat[nonmissing_indices] = np_array.flat[nonmissing_indices] 

In [7]:
## Check approprate rank that can be used for low rank model estimation 
## Using singular values in the svd of the data matrix 

_, s, _ = np.linalg.svd(rating_matrix_ini) 
print(s)

[3.24624479e+05 8.56280630e+04 4.84166579e+04 4.29315980e+04
 1.41209999e+04 8.56729200e+03 5.09323163e+03 2.67514353e+03
 2.14319199e+03 1.99084400e+03 1.66307602e+03 1.57979801e+03
 1.19670910e+03 1.15292219e+03 8.40011427e+02 7.77785156e+02
 6.36125114e+02 6.21344575e+02 5.06515136e+02 4.68598552e+02
 4.27068121e+02 3.76699983e+02 3.16652501e+02 2.85517793e+02
 1.57022667e+02 1.48204131e+02 1.38011386e+02 1.32177616e+02
 1.24163461e+02 1.21864473e+02 1.00556217e+02 8.76294065e+01
 8.56844247e+01 7.93654509e+01 7.85326821e+01 7.28420212e+01
 6.37270183e+01 6.17567338e+01 5.75570954e+01 5.64826668e+01
 4.09477164e+01 4.02473642e+01 3.88869254e+01 3.79045148e+01
 3.25519534e+01 3.21795481e+01 2.96245538e+01 2.80120425e+01
 2.58282293e+01 2.36422223e+01 2.34419493e+01 2.28912864e+01
 1.95005289e+01 1.51034947e+01 1.40411651e+01 1.00869849e+01]


In [8]:
RANK = 14 

In [9]:
## Impute missing values with column mean to begin with 
col_mean = np.nanmean(np_array, axis=0)
inds = np.where(np.isnan(np_array))
np_array[inds] = np.take(col_mean, inds[1]) 

In [10]:
def fit_low_rank_model(rank, 
                       rating_matrix_ini, 
                       train_ind, 
                       train_data, 
                       n_iter, 
                       convergence_thresh, 
                       verbose, 
                       data1=None, 
                       missing_indices=None): 
    
    """Fit the low rank model. 
    Return the estimation of the low rank model - (n_movies * n_users) matrix

    Keyword arguments:
    rank -- the rank of low rank model
    rating_matrix_ini -- imputed initialization
    train_ind -- index of training data
    train_data -- ratings of training set
    n_iter -- the max number of iterations
    convergence_thresh -- the threshold of convergence to 0
    """
    
    previous_fitting_error = 100
    # Initialization
    low_rank_estimate = np.zeros(rating_matrix_ini.shape) - 1 
    # fill input data
    low_rank_estimate.flat[train_ind] = train_data
    # get the indexes of missing data
    missing_inds = np.where(low_rank_estimate.flat == -1)
    # fill missing data with imputed values
    low_rank_estimate.flat[missing_inds] = rating_matrix_ini.flat[missing_inds]

    
    for ind in range(n_iter):
        # Updates
        low_rank_estimate.flat[train_ind] = train_data
        u, s, v = np.linalg.svd(low_rank_estimate)
        s_matrix = s  * np.eye(len(s))
        low_rank_estimate = np.matmul(np.matmul(u[:,0:rank], s_matrix[0:rank,0:rank]), v[0:rank,:])
        # Compute error
        fitting_error = np.sqrt(((train_data - low_rank_estimate.flat[train_ind])**2).mean())
        if (not (data1 == None)): 
            #true fitting error, compared to true values
            true_fitting_error = np.sqrt(((data1.flat[missing_indices] - low_rank_estimate.flat[missing_indices])**2).mean())
        if verbose:
            print("Iteration " + str(ind) + " Error: " + str(fitting_error))
            if (not (data1 == None)): 
                print("Iteration " + str(ind) + " True Error: " + str(true_fitting_error)) 
                print() 
        
        # Stopping criterion
        if (fitting_error <= convergence_thresh):
            print('converged, breaking')
            break
    return low_rank_estimate

n_iter = 50 
convergence_thresh = 1e-4
verbose = True
rank = RANK 
train_data = np_array.flat[nonmissing_indices] 
# estimate =fit_low_rank_model(rank,rating_matrix_ini,nonmissing_indices,
#         train_data,n_iter,convergence_thresh,verbose, nonmissing_data, MCAR_missing_indices)

estimate = fit_low_rank_model(rank, 
                             rating_matrix_ini, 
                             nonmissing_indices, 
                             train_data, 
                             n_iter, 
                             convergence_thresh, 
                             verbose) 

print("Estimate = \n", estimate)



Iteration 0 Error: 3.354633929663363
Iteration 1 Error: 3.295053173064625
Iteration 2 Error: 3.275290263601959
Iteration 3 Error: 3.263980044805593
Iteration 4 Error: 3.2558335263965095
Iteration 5 Error: 3.2490843354052354
Iteration 6 Error: 3.2430448606154982
Iteration 7 Error: 3.2374216171378154
Iteration 8 Error: 3.2320735924278745
Iteration 9 Error: 3.226918844284466
Iteration 10 Error: 3.2218982149201616
Iteration 11 Error: 3.216962813375357
Iteration 12 Error: 3.212071385142291
Iteration 13 Error: 3.207191312928791
Iteration 14 Error: 3.202300412790329
Iteration 15 Error: 3.197388324252124
Iteration 16 Error: 3.19245703222047
Iteration 17 Error: 3.187520338076573
Iteration 18 Error: 3.18260219083432
Iteration 19 Error: 3.177733871752386
Iteration 20 Error: 3.172950181775079
Iteration 21 Error: 3.168285006840396
Iteration 22 Error: 3.1637668559449317
Iteration 23 Error: 3.1594150752058834
Iteration 24 Error: 3.155237362454829
Iteration 25 Error: 3.151228945540961
Iteration 26 Err

In [11]:
estimate 

array([[ 6.61962684e+01,  1.81973668e+00,  3.48616681e+00, ...,
         1.66796334e+02, -8.64531161e-02,  2.89723496e+01],
       [ 5.17189879e+01,  5.18385723e-01,  2.66403214e+00, ...,
         1.69910780e+02,  5.79272884e-02,  3.00620125e+01],
       [ 7.41686508e+01,  1.63052759e+00,  3.64700243e+00, ...,
         1.26246504e+02, -2.34994229e-01,  2.91360860e+01],
       ...,
       [ 8.06312798e+01,  1.31348998e+00,  3.06616356e+00, ...,
         1.57091072e+02, -7.62942989e-03,  2.41935927e+01],
       [ 2.74708626e+01,  1.12260574e+00,  2.86345915e+00, ...,
         1.90327529e+02,  1.52103200e-01,  2.38274392e+01],
       [ 4.26737626e+01,  1.54681881e+00,  3.56502306e+00, ...,
         1.51892120e+02, -8.94782855e-02,  3.05768693e+01]])

In [12]:
with open('lowrank_estimate.npy', 'wb') as f:
    np.save(f, estimate)